In [1]:
"""In this lab, the objective is to identify the customers who were active in both May and June, and how did their activity differ between months. To achieve this, follow these steps:

Establish a connection between Python and the Sakila database."""
    
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector

In [3]:
from getpass import getpass

In [4]:
password = getpass()

In [5]:
# Create the connection

conn = mysql.connector.connect(host = 'localhost',
                               user = 'root',
                               password = password)

In [6]:
conn.is_connected()

True

In [7]:
# Create a cursor that will hold our query, execute it and fetch it (is takes info from python to sql and brings it back)

cursor = conn.cursor()

In [9]:
import pandas as pd

Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.

month: an integer representing the month for which rental data is to be retrieved.

year: an integer representing the year for which rental data is to be retrieved.

The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [26]:
def rentals_month(month, year):
    conn = mysql.connector.connect(host = 'localhost',
                               user = 'root',
                               password = password)
    cursor = conn.cursor()
    query = f"""
            SELECT *
            FROM sakila.rental
            WHERE MONTH(rental_date) = {month}
            AND YEAR(rental_date) = {year};
            """
    cursor.execute(query)
    results = cursor.fetchall()
    df = pd.DataFrame(results, columns=cursor.column_names)
    return df
rentals_month(5, 2005)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53


Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()

In [35]:
def rentals_count (month, year):
    conn = mysql.connector.connect(host = 'localhost',
                               user = 'root',
                               password = password)
    cursor = conn.cursor()
    query = f"""SELECT customer_id,COUNT(*) AS number_of_rentals 
                FROM sakila.rental
                WHERE MONTH(rental_date) = {month}
                AND YEAR(rental_date) = {year}
                GROUP BY customer_id"""
    cursor.execute(query)
    results = cursor.fetchall()
    new_df = pd.DataFrame(results, columns=cursor.column_names)
    #grouped = new_df.groupby('customer_id')
    return new_df
rentals_count(5, 2005)

,customer_id,number_of_rentals
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


In [44]:
rental_data = rentals_month(5, 2005)

def rental_count_month(rental_data, month, year):
    rental_counts = rental_data.groupby('customer_id').size().reset_index(name='rental_count')
    column_name = f"rentals_{month:02d}_{year}"
    result_df = pd.DataFrame(rental_counts, columns=['customer_id', 'rental_count'])
    result_df.rename(columns={'rental_count': column_name}, inplace=True)
    return result_df
rental_count_month(rental_data, 5, 2005)

,customer_id,rentals_05_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. 

The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [49]:
# Those 2 last exercises I couldn't have done without the help of chatgpt. I'm not even sure that I understand them reading the already done code. Very far from my understanding :/

rental_data_1 = rentals_month(5, 2005)
rental_data_2 = rentals_month(2, 2006)
def compare_rentals(rental_data_1, rental_data_2, month1, year1, month2, year2):
    rental_counts_1 = rental_data_1.groupby('customer_id').size().reset_index(name='rental_count_1')
    rental_counts_2 = rental_data_2.groupby('customer_id').size().reset_index(name='rental_count_2')
    merged_counts = pd.merge(rental_counts_1, rental_counts_2, on='customer_id', how='outer')
    merged_counts['difference'] = merged_counts['rental_count_1'].fillna(0) - merged_counts['rental_count_2'].fillna(0)
    return merged_counts

compare_rentals(rental_data_1, rental_data_2, 5, 2005, 2, 2006)

,customer_id,rental_count_1,rental_count_2,difference
0,1,2.0,NaN,2.0
1,2,1.0,NaN,1.0
2,3,2.0,NaN,2.0
3,5,3.0,1.0,2.0
4,6,3.0,NaN,3.0
...,...,...,...,...
534,527,NaN,1.0,-1.0
535,568,NaN,1.0,-1.0
536,579,NaN,1.0,-1.0
537,585,NaN,1.0,-1.0
